In [ ]:
#!pip install histocartography

In [ ]:
import dgl
import numpy as np
import pandas as pd
import torch
import copy
import cgm_model
import importlib
importlib.reload(cgm_model)
import cgm_model

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# DATA_PATH = '/content/drive/Shareddrives/BIOINF 593 Course Project/'
DATA_PATH = './data/'
pt_data = pd.read_excel(DATA_PATH+'CRC_TMAs_patient_annotations.xlsx')
cell_data = pd.read_csv(DATA_PATH+'CRC_master.csv')

In [ ]:
gnn_params = {'readout_op': 'concat',
 'layer_type': 'gin_layer',
 'agg_type': 'mean',
 'output_dim': 16,
 'num_layers': 2,
 'readout_type': 'mean',
 'batch_norm': True,
 'act': 'relu'}
classification_params = {'hidden_dim': 16, 'num_layers': 2}

In [ ]:
data, labs, ids = cgm_model.data_to_dgl_graph(pt_data, cell_data, mode='connectivity', normalize=True)

In [ ]:
num_models = 10
cgm_reps = []
for _ in range(num_models):
    model = cgm_model.CGModel(gnn_params=gnn_params, classification_params=classification_params, node_dim=74, num_epochs=50)
    model.train(data)
    cgm_reps.append(copy.deepcopy(model))

In [ ]:
res = cgm_model.get_model_score_dict(cgm_reps, data, labs)

In [ ]:
len(res)

In [ ]:
import pickle

In [ ]:
with open("importance_scores_group_prediction.pkl", 'wb') as f:
    pickle.dump(res, f)

In [ ]:
scl = res[0]

In [ ]:
import matplotlib.pyplot as plt
for ct in scl.cell_type.unique():
    plt.hist(scl[scl['cell_type'] == ct].score_campp)
    plt.title(ct)
    plt.show()